In [ ]:
""" Задачи """

Задача 1. Оценить необходимый размер групп

Допустим, мы хотим провести эксперимент, в который попадают клиенты, совершившие покупку во время эксперимента.

- Метрика — средняя выручка с пользователя за время эксперимента;
- Продолжительность — одна неделя;
- Уровень значимости — 0.05;
- Допустимая вероятность ошибки II рода — 0.1;
- Ожидаемый эффект — 20 рублей.

Оцените необходимый размер групп по данным о покупках за неделю с 21 по 28 февраля.

In [5]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import norm, ttest_ind
import os

In [2]:
def read_file(file_name):
    if not os.path.exists(file_name):
        raise FileNotFoundError(f'Файл {file_name} не найден')
    return pd.read_csv(file_name)

In [3]:
df_sales = read_file('df_sales.csv')

In [ ]:
#функия нахождения размера выборки для эксперимента
def get_sample_size_abs(epsilon, std, alpha=0.05, beta=0.2):
    t_alpha = norm.ppf(1 - alpha / 2, loc=0, scale=1)
    t_beta = norm.ppf(1 - beta, loc=0, scale=1)
    z_scores_sum_squared = (t_alpha + t_beta) ** 2
    sample_size = int(
        np.ceil(
            z_scores_sum_squared * (2 * std ** 2) / (epsilon ** 2)
        )
    )
    return sample_size

In [9]:
df_sales['date'] = pd.to_datetime(df_sales['date'])

date_start = pd.to_datetime('2022-02-21')
date_end = pd.to_datetime('2022-02-28')

df = df_sales[(df_sales['date']>=date_start) & (df_sales['date']<date_end)]

In [13]:
df.head(3)

,sale_id,date,count_pizza,count_drink,price,user_id
63207,1063208,2022-02-21 10:02:02,4,1,3030,fcaa0f
63208,1063209,2022-02-21 10:04:43,3,1,2250,8d8445
63209,1063210,2022-02-21 10:05:40,1,0,780,e21af3


In [30]:
df.shape

(25347, 6)

In [ ]:
#находим сумму покупок по каждому клиенту
df_sum = df.groupby('user_id')['price'].agg('sum').reset_index()
#считаем стандартное отклонение
std = df_sum['price'].std()
#подставляем все необходимые данные в функцию 
answer = get_sample_size_abs(20, std, 0.05, 0.1)

print(round(answer, -1))

34570


Задача 2. В прошлом задании получилось, что необходимый размер групп больше имеющихся данных за одну неделю. 

Какой минимальный эффект мы можем отловить с теми же вероятностями ошибок на данных с 21 по 28 февраля?

In [32]:
#функция для оценки размера выборки
def get_minimal_determinable_effect(std, sample_size, alpha=0.05, beta=0.2):
    t_alpha = norm.ppf(1 - alpha / 2, loc=0, scale=1)
    t_beta = norm.ppf(1 - beta, loc=0, scale=1)
    disp_sum_sqrt = (2 * (std ** 2)) ** 0.5
    mde = (t_alpha + t_beta) * disp_sum_sqrt / np.sqrt(sample_size)
    return mde

In [ ]:
#считаем размер выборки на исторических данных, которую можем получить
sample_size = np.ceil(df.shape[0]/2)

In [ ]:
#счиатем MDE
mde = get_minimal_determinable_effect(std, sample_size, 0.05, 0.1)

print(round(mde, 0))

33.0
